In [1]:
cd ../

C:\Users\Ko\Documents\Ko\Clean\Change_Detection\unet-master


#### 1. Package import

In [9]:
# from model import *
from data import *

---

#### 2. Unet model architecture

In [3]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

def unet(pretrained_weights = None,input_size = (256,256,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 2e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    model.summary()

#     if(pretrained_weights):
#         model.load_weights(pretrained_weights)

    return model


---

#### 3. Data augmentation(데이터 증식) 및 generator 생성을 통한 학습

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(12,'data/land/train','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_land(1)_2.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=100,epochs=100,callbacks=[model_checkpoint])

In [5]:
data_gen_args = dict(rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(16,'data/land/train','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_land.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=100,epochs=100,callbacks=[model_checkpoint])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_24[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_25[0][0]                  
____________________________________________________________________________________________

Epoch 1/100
100/100 [==============================] - 72s 627ms/step - loss: 0.4819 - accuracy: 0.8918

Epoch 00001: loss improved from inf to 0.36949, saving model to unet_land.hdf5
Epoch 2/100
100/100 [==============================] - 60s 599ms/step - loss: 0.2635 - accuracy: 0.8931

Epoch 00002: loss improved from 0.36949 to 0.24452, saving model to unet_land.hdf5
Epoch 3/100
100/100 [==============================] - 60s 604ms/step - loss: 0.2148 - accuracy: 0.9030

Epoch 00003: loss improved from 0.24452 to 0.20783, saving model to unet_land.hdf5
Epoch 4/100
100/100 [==============================] - 61s 615ms/step - loss: 0.2088 - accuracy: 0.9039

Epoch 00004: loss improved from 0.20783 to 0.19705, saving model to unet_land.hdf5
Epoch 5/100
100/100 [==============================] - 61s 610ms/step - loss: 0.1797 - accuracy: 0.9203

Epoch 00005: loss improved from 0.19705 to 0.17427, saving model to unet_land.hdf5
Epoch 6/100
100/100 [==============================] - 60s 604ms

Epoch 47/100
100/100 [==============================] - 61s 616ms/step - loss: 0.0331 - accuracy: 0.9857

Epoch 00047: loss improved from 0.03361 to 0.03312, saving model to unet_land.hdf5
Epoch 48/100
100/100 [==============================] - 62s 629ms/step - loss: 0.0327 - accuracy: 0.9860

Epoch 00048: loss improved from 0.03312 to 0.03235, saving model to unet_land.hdf5
Epoch 49/100
100/100 [==============================] - 63s 627ms/step - loss: 0.0326 - accuracy: 0.9859

Epoch 00049: loss improved from 0.03235 to 0.03211, saving model to unet_land.hdf5
Epoch 50/100
100/100 [==============================] - 63s 636ms/step - loss: 0.0314 - accuracy: 0.9865

Epoch 00050: loss improved from 0.03211 to 0.03158, saving model to unet_land.hdf5
Epoch 51/100
100/100 [==============================] - 62s 627ms/step - loss: 0.0314 - accuracy: 0.9865

Epoch 00051: loss did not improve from 0.03158
Epoch 52/100
100/100 [==============================] - 63s 623ms/step - loss: 0.0313 - acc

100/100 [==============================] - 62s 620ms/step - loss: 0.0374 - accuracy: 0.9846

Epoch 00095: loss did not improve from 0.02412
Epoch 96/100
100/100 [==============================] - 62s 623ms/step - loss: 0.0304 - accuracy: 0.9875

Epoch 00096: loss did not improve from 0.02412
Epoch 97/100
100/100 [==============================] - 62s 618ms/step - loss: 0.0254 - accuracy: 0.9891

Epoch 00097: loss did not improve from 0.02412
Epoch 98/100
100/100 [==============================] - 62s 616ms/step - loss: 0.0238 - accuracy: 0.9897

Epoch 00098: loss improved from 0.02412 to 0.02374, saving model to unet_land.hdf5
Epoch 99/100
100/100 [==============================] - 61s 617ms/step - loss: 0.0232 - accuracy: 0.9900

Epoch 00099: loss improved from 0.02374 to 0.02335, saving model to unet_land.hdf5
Epoch 100/100
100/100 [==============================] - 62s 617ms/step - loss: 0.0239 - accuracy: 0.9897

Epoch 00100: loss did not improve from 0.02335


---

#### 4. Test 데이터에 대한 예측

In [6]:
testGene = testGenerator("data/land/test", target_size = (448, 784))
model = unet()
model.load_weights("unet_land.hdf5")
results = model.predict_generator(testGene,2,verbose=1)
# saveResult("data/land/test",results)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 256, 256, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_48[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_49[0][0]                  
____________________________________________________________________________________________

C:\Users\Ko\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2/2 [==============================] - 3s 64ms/step


In [7]:
saveResult("data/land/test",results)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
